# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4880, uptime 0:00:33
mini_beamline                    RUNNING   pid 4881, uptime 0:00:33
random_walk                      RUNNING   pid 4882, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4883, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4884, uptime 0:00:33
simple                           RUNNING   pid 4885, uptime 0:00:33
thermo_sim                       RUNNING   pid 4886, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4887, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-19 00:50:34
Persistent Unique Scan ID: '9f19c359-a70e-4a86-a205-a40559e29d43'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 00:50:35.1 |     -1.000 |      93332 |
|         2 | 00:50:35.1 |     -0.500 |      95394 |
|         3 | 00:50:35.2 |      0.000 |      96524 |
|         4 | 00:50:35.2 |      0.500 |      96229 |
|         5 | 00:50:35.2 |      1.000 |      94662 |


+-----------+------------+------------+------------+
generator scan ['9f19c359'] (scan num: 3)





('9f19c359-a70e-4a86-a205-a40559e29d43',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/9f19c359-a70e-4a86-a205-a40559e29d43-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-19 00:50:35
Persistent Unique Scan ID: '3cd29bb6-7892-40d5-beb6-440755174bca'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 00:50:35.7 |          -1 | 1082907632 |


|         2 | 00:50:35.8 |          -0 | 1096384269 |


|         3 | 00:50:35.9 |           0 | 1103352150 |


|         4 | 00:50:36.0 |           0 | 1110501178 |


|         5 | 00:50:36.1 |           1 | 1116581214 |


+-----------+------------+-------------+------------+
generator scan ['3cd29bb6'] (scan num: 4)





('3cd29bb6-7892-40d5-beb6-440755174bca',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-19 00:50:36
Persistent Unique Scan ID: 'ec83fa45-7755-4cc2-bd90-7f2c157748d7'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 00:50:36.7 | 1130282341 |


|         2 | 00:50:37.6 | 1099765963 |


|         3 | 00:50:38.6 | 1053855263 |


...

|        28 | 00:51:03.6 | 1083313329 |


|        29 | 00:51:04.6 | 1131553919 |


|        30 | 00:51:05.6 | 1096771128 |


+-----------+------------+------------+
generator count ['ec83fa45'] (scan num: 5)





('ec83fa45-7755-4cc2-bd90-7f2c157748d7',)